In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms
print("PyTorch Version:",torch.__version__)

PyTorch Version: 1.3.0.post2


# 在做图片识别的时经常会用到Batch Norm
![Image1](../common/images/5-pytorch01.png) 

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=20,kernel_size=5,stride=1)
        self.conv2 = nn.Conv2d(20,50,5,1)
        self.fc1 = nn.Linear(4*4*50,500) # 池化后的第一层全连接
        self.fc2 = nn.Linear(500,10) # 第二层全连接:0-9的10种数字的分类
    
    def forward(self,x):
        # x 是一张1*28*28的手写数字
        x = F.relu(self.conv1(x)) # [24*24]：28-5+1=24
        x = F.max_pool2d(x,kernel_size=2,stride=2) # [12*12]：(24-2)/2 +1 =12
        x = F.relu(self.conv2(x)) # [8*8]: 12-5+1 =8
        x = F.max_pool2d(x,2,2) # [4*4]: 
        # reshape一下
        x = x.view(-1,4*4*50)
        x = self.fc1(x)
        x = self.fc2(x)
        
        return F.log_softmax(x,dim=1)
        

In [3]:
mnist_data = datasets.MNIST(root="/Users/zhenwuzhou/AiProject/data/mnist_data",train=True, download=True,
                            transform=transforms.Compose([
                                transforms.ToTensor(), # 把图片转换成tensor
                            ]))
mnist_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /Users/zhenwuzhou/AiProject/data/mnist_data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
           )

In [4]:
mnist_data[233][0].shape

torch.Size([1, 28, 28])

In [28]:
np.mean([2.,3.,4.])  # 计算平均值操作

3.0

In [25]:
# 把mnist_data中的图片（d[0]是图片，d[1]是标签）都拿出来转换成numpy
data = [d[0].data.cpu().numpy() for d in mnist_data]
np.mean(data) # 计算出所有像素的平均值

0.13066062

In [30]:
np.std([2.,3.,4.]) # 是标准差，方式开根号，方差是2/3 = 0.6666666

0.816496580927726

In [29]:
np.std(data) # 标准差

0.30810776

In [34]:
def train(model,device,train_loader,optimizer,epoch):
    model.train()
    for idx,(data,target) in enumerate(train_loader):
        data,target = data.to(device), target.to(device)
        
        pred = model(data) # batch_size*10
        
        # 定义代价函数：target是一个数字，这个loss就是直接把target对应数字上的那个pred的值取出来;
        # 因为我们的模型返回的log_softmax(x)结果，相当于把0-1的那个概率值用指数函数log了一下
        # 损失值是[-inf, 0)
        # a Tensor of the same dimension and shape as the input with values in the range [-inf, 0)
        loss = F.nll_loss(pred,target) 
        
        # 反向传播backward
        optimizer.zero_grad()
        loss.backward()
        
        #GD
        optimizer.step()
        
        if idx % 100 ==0:
            print("Train Epoch：{}, iteration:{},Loss:{}".format(epoch,idx,loss.item()))
        

In [52]:
def test(model,device,test_loader):
    model.eval()
    total_loss = 0.
    correct = 0.
    with torch.no_grad():
        for idx,(data,target) in enumerate(test_loader):
            data,target = data.to(device), target.to(device)
        
            pred = model(data) # batch_size*10
            total_loss = F.nll_loss(pred,target,reduction="sum").item() 
            
            correctPred = pred.argmax(dim=1) # batch_size*1
            correct += correctPred.eq(target.view_as(correctPred)).sum().item() # 计算正确数
            
    model.train()
        
    total_loss /= len(test_loader.dataset)
    acc = correct/len(test_loader.dataset)*100
    print("Test loss：{}, Accuracy:{}".format(total_loss,acc))

In [53]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 32
train_dataloader = torch.utils.data.DataLoader(
    datasets.MNIST("/Users/zhenwuzhou/AiProject/data/mnist_data",train=True,download=False,
                  transform=transforms.Compose([
                      transforms.ToTensor(),
                      transforms.Normalize(mean=(0.1307,),std=(0.3081,)),#对像素进行标准化
                  ])),
    batch_size = batch_size,shuffle=True,
    num_workers = 1,pin_memory=True
)

test_dataloader = torch.utils.data.DataLoader(
    datasets.MNIST("/Users/zhenwuzhou/AiProject/data/mnist_data",train=False,download=False,
                  transform=transforms.Compose([
                      transforms.ToTensor(),
                      transforms.Normalize(mean=(0.1307,),std=(0.3081,))
                  ])),
    batch_size = batch_size,shuffle=True,
    num_workers = 1,pin_memory=True
)

lr =0.01
momentum = 0.5
model = Net().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=lr,momentum=momentum)

num_epochs = 2
for epoch in range(num_epochs):
    train(model,device,train_dataloader,optimizer,epoch)
    test(model,device,test_dataloader)

torch.save(model.state_dict(),"/Users/zhenwuzhou/.keras/models/PytrochCNN/1model_dict.pth")
torch.save(model,"/Users/zhenwuzhou/.keras/models/PytrochCNN/1model.pth")

Train Epoch：0, iteration:0,Loss:2.3033761978149414
Train Epoch：0, iteration:100,Loss:0.5190887451171875
Train Epoch：0, iteration:200,Loss:0.2233976423740387
Train Epoch：0, iteration:300,Loss:0.17817017436027527
Train Epoch：0, iteration:400,Loss:0.14404480159282684
Train Epoch：0, iteration:500,Loss:0.10768669098615646
Train Epoch：0, iteration:600,Loss:0.14255644381046295
Train Epoch：0, iteration:700,Loss:0.09527770429849625
Train Epoch：0, iteration:800,Loss:0.4986037611961365
Train Epoch：0, iteration:900,Loss:0.22145932912826538
Train Epoch：0, iteration:1000,Loss:0.057534463703632355
Train Epoch：0, iteration:1100,Loss:0.026500415056943893
Train Epoch：0, iteration:1200,Loss:0.26053687930107117
Train Epoch：0, iteration:1300,Loss:0.3006894290447235
Train Epoch：0, iteration:1400,Loss:0.1352003812789917
Train Epoch：0, iteration:1500,Loss:0.09746439009904861
Train Epoch：0, iteration:1600,Loss:0.08620873093605042
Train Epoch：0, iteration:1700,Loss:0.14847563207149506
Train Epoch：0, iteration:1

KeyboardInterrupt: 